***
<center><h1>Face Rhythm</h1></center>

***

<img src="https://drive.google.com/uc?export=view&id=14bN4PKuwZQORI_A1wK9r6ZV4_QpZL_L2">

This notebook calculates Face Rhythm. It's dope

***

##### Notebook Shortcuts
- **[Notebook Setup](#Notebook-Setup)**: Prepare all the necessary config files and folders
- **[Set ROI](#Set-ROI)**: Set the ROI for the analysis
- **[Run Optic Flow](#Run-Optic-Flow)**: Run the optic flow analysis
- **[Clean Optic Flow](#Clean-Optic-Flow)**: Optic flow post-processing
- **[Analysis](#Analysis)**: Decompose and Analyze the optic flow data in many ways

***

# Tips on running this notebook:
In theory it would be nice if you could just enter the path of the video(s) and just let it run all the way through. In practice, there are a few hoops to jump through
- Run the import cell block (two blocks below this one). This should pretty much always be done, even if you are loading precomputed file from disk instead of calculating them. This step loads in some useful meta data used throughout.
- Use the Save and Load cell blocks to save and load data after long calculations. It seriously helps with iterating, debugging, and memory allocation.
    - These arrays can be BIG. I regularly go past 128 GB when running my data through. It's often easiest to just restart the kernel, load in just the precomputed variables needed and run the script in the middle (skipping all the previous computations)
- There are two cell blocks for calculating the optic flow (parameters set independently as well), one single-threaded and one multi-threaded. Do parameter tuning on the single-threaded one so you can quit out of it, as well as watch the calculation as it happens with showVideo_pref=1. The multi-threaded one is only faster if you have a lot of cores in your CPU (>10), then it's faster, else stick with the single-threaded version and set showVideo_pref=0.
- Parameter tuning should be pretty easy. Most of the cell blocks can be halted in the middle of computation so you can just look at a small bit of the data, tune a parameter and then rerun the code

### The most important parameters:  
***(Consider all of these before you run the code for the first time)***
- Optic flow params:
    - **'spacing'**: ~ 3 to 12. Spacing between dots, in pixels. Inversely related to number of dots to use in the calculation. Try to keep the number of dots below 2000 if possible (eats up memory and computation time). More dots generally means better final results, more robust to outliers and weird stuff. I'd make the spacing as small (more dots) as you can go before you run out of RAM in the final calculations
    - **lk_params 'win_size'**: ~ 25,25 to 80,80. This is the spatial integration window for the optical flow measurement. Try to make it as small as possible without it becoming unstable. The two values are for X and Y length of square integration window. Probably keep the same for most applications
- Outlier removal params:
    - **outlier_threshold_positions**: ~ 20 to 100. If a dot strays more than this many pixels away from its anchor position, its displacement in the dimension it cross the threshold in, for those time points (and some time points around it, see params below), for that dot only, will be set to zero
    - **outlier_threshold_displacements** ~ 5 to 25. Similar to above, but for displacement. Only the outlier time points are removed (no window around outliers considered).
    - **framesHalted_beforeOutlier**: ~ 0 to 30. The number of frames to also remove before detected outlier events. Consider what is causing your outlier event. If it is an arm movement or something, how long does such a movement last? How long before it will cause a dot to move to the outlier threshold?
    - **framesHalted_afterOutlier**: ~ 0 to 10. Simlar to above but for after an outlier event is detected
    - **relaxation_factor** : ~ 0.03 to 0.1. This is the rate of the exponential decay / relaxation / attraction back to the anchor position that a point undergoes. It is meant to prevent baseline drift. Think of it like a high pass on the dot position trace
- Spectral analysis params:
    - **win_len**: ~ 0.1 to 1.0. The length of the time window used for the short-time Fourier transform. Longer gives better spectral resolution, shorter gives better temporal resolution. There are several other parameters that are related but this is the most important. Longer windows (along with decreasing the overlap parameter) also decrease the size of the output spectrograms, which can help with memory and computation time in the subsequent analyses
- TCA:
    - **rank = 6**: ~ 2 to 10. The number of factors to look for in the PARAFAC model. More can be good but less reproduceable, but less can mix together obviously different factors

In [ ]:
# widen jupyter notebook window
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [ ]:
# Import libraries
# The only non-publicly available function is mtaper_specgram

import sys
import os
import time
import gc
import copy
import math

import cv2
import imageio

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import numpy as np
import numpy.matlib

import scipy
import scipy.signal

import sklearn as sk
import sklearn.decomposition
import skimage.draw

import torch
import torch.cuda

import multiprocessing
from multiprocessing import Pool

import librosa

In [ ]:
#sys.path.insert(0, '/media/rich/Home_Linux_partition1/github_repos/tensorly')

import tensorly as tl
import tensorly.decomposition
import tensorly.random

***
<center><h1>Notebook Setup</h1></center>

***

### Creates config and locates videos

As always, make sure to read through the config parameters before running.

In [ ]:
# Create config and check the code versioning
from face_rhythm.util import helpers
from pathlib import Path
import ruamel.yaml as yaml

base_dir = Path('./').resolve()
config_filepath = base_dir / 'config.yaml'
 
helpers.generate_config(config_filepath) 
helpers.version_check(config_filepath)

In [ ]:
from face_rhythm.util import helpers

### IMPORT VIDEOS
## Define DIRECTORY of video(s) to use and IMPORT videos (as read objects) into openCV
## Be careful to follow input the directories properly below. Input directory
## and file name (or file name prefix) in either the m

# This option imports all of the videos with a defined file name prefix in a folder
# OR just imports a single defined file
config = helpers.load_config(config_filepath)

config['multiple_files_pref'] = False
config['dir_vid'] = str(Path('../data/').resolve())

# Used only if 'multiple_files_pref'==1
config['fileName_vid_prefix'] = 'cam3_2020-11-02-185732-' 
config['fileName_vid_numDigitsInIteration'] = 4 # number of digits in the movie (used when movie is broken up into chunks)
config['fileName_vid_suffix'] = '.avi'

# Used only if 'multiple_files_pref'==0
config['fileName_vid'] = 'raw/gmou06_082720_faceTrack_session1.avi'


### == IMPORT videos ==
config['print_fileNames_pref'] = 1

helpers.save_config(config, config_filepath)

helpers.import_videos(config_filepath)
h# Step 1: Set Up

### Creates config and locates videos

As always, make sure to read through the config parameters before running.elpers.get_video_data(config_filepath)

***
<center><h1>Set ROI</h1></center>

***

### Manually specify your roi

This is good if your animal doesn't fill the view and if you have stationary objects nearby.

In [ ]:
from face_rhythm.util import set_roi

### Select POLYGON SUBFRAME for DISPLACEMENT Eignfaces
## This block of code will pop up a little GUI. Click around the
## region of the face that you want to include in the analysis.
## When you are done, press enter twice to accept and exit the GUI.

config = helpers.load_config(config_filepath)
config['vidToSet'] = 1 # 1 indexed. Sets the video to use to make an image
config['frameToSet'] = 2 # 1 indexed. Sets the frame number to use to make an image
config['save_dir'] = str(Path('../data/processed').resolve())
helpers.save_config(config, config_filepath)

pts_all = set_roi.roi_workflow(config_filepath)

In [ ]:
config = helpers.load_config(config_filepath)
save_name = f'pts_all'
save_fullPath = f'{config['save_dir']}/{save_name}.npy'
np.save(save_fullPath , pts_all)
config['path_pts_all'] = save_fullPath
helpers.save_config(config, config_filepath)

***
<center><h1>Run Optic Flow</h1></center>

***

### Run as either mono or multi threaded depending on run time and number of dots

Multithread may struggle when too many dots are selected (memory overload)

In [ ]:
from face_rhythm.optic_flow import optic_flow

### IMPORT and CALCULATE DISPLACEMENT FIELD
### __SINGLE(ish) THREAD__ VERSION

### Calculate DISPLACEMENT FIELD using dot grid within subframe
## The only thing coming out of the code block that matters is the 'displacements' variable

## I use imageio (ffmpeg) because openCV doesn't seem to import as many frames as imageio does (wtfffff), 
## and I can't preallocate because openCV and imageio give inaccurate total frame numbers from metadata (WTFFFF).

## Important assumptions about using this code verses the single (ish) threaded version:
## 1. numFrames (per file) + 1000 (USING OPENCVs cv2.CAP_PROP_FRAME_COUNT) must be > true number of frames
##  as found using the imageio import method
## 2. Debugging is hard. If you interrupt the kernel while it's doing the parallel pool, the kernel is kind of fucked
##  and generally requires a restart
## 3. I haven't figured out how to track progress. I know there are probably ways to make wait bars, but I'll leave that
##  to a software engineer.

### == PREFERENCES ==
config = helpers.load_config(config_filepath)

config['vidNums_toUse'] = range(config['numVids']) ## 0 indexing
config['spacing'] = 3  ## This is the distance between points in the grid (both in x and y dims)

config['showVideo_pref'] = False  ## much faster when video is off
config['dot_size'] = 1  ## for viewing purposes

## below will print the fps ever 'fps_counterPeriod' . Useful for checking the speed of import.
## Best to turn off when doing a full run. (this is mostly for optimizing and debugging)
config['printFPS_pref'] = False
config['fps_counterPeriod'] = 10 ## number of frames to do a tic toc over

## Parameters for lucas kanade optical flow
## win size: spatial integration window (make small as possible, but make bigger if youre having issues with outliers)
## max level: only moderate effects if everything working properly. Keep around 3.
## criteria values have to do with the search algorithm. For speed: EPS small, COUNT big.
config['lk_winSize']  = (35,35)
config['lk_maxLevel'] = 4
config['lk_criteria']    = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100, 0.001)

config['optic_multithread'] = True

helpers.save_config(config, config_filepath)


### == CALCULATION ==

tic = time.time()

displacements, pointInds_toUse = optic_flow.optic_workflow(config_filepath)

toc = time.time() - tic

    
print(f'\n ===== Displacement calculation completed! =====')
print(f'Total elapsed time: {round(toc/(60*60) , 2)} hours')
print(f'Average frames per second: {round(numFrames_total/toc , 2)} fps')

In [ ]:
## This cell block is here just because sometimes its nice to cancel early out of the iterative loading (above cell block) and just play with a short trace. This block cleans out the NaNs
displacements = displacements[:,:,~np.isnan(displacements[0,0,:])]  # this removes the trailing and interleaved NaNs in the arrays

In [ ]:
config = helpers.load_config(config_filepath)

save_dir = config['save_dir']

save_name = f'pointInds_toUse'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , pointInds_toUse)
config['path_pointsInds_toUse'] = save_fullPath

save_name = f'displacements'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , displacements)
config['path_displacements'] = save_fullPath

helpers.save_config(config, config_filepath)

# save_name = f'params_video'
# save_fullPath = f'{save_dir}/{save_name}.npy'
# np.save(save_fullPath , params_video)

# save_name = f'Fs'
# save_fullPath = f'{save_dir}/{save_name}.npy'
# np.save(save_fullPath , Fs)

In [ ]:
config = helpers.load_config(config_filepath)


displacements = np.load(config['path_displacements'])
pointInds_toUse = np.load(config['path_pointsInds_toUse'])

***
<center><h1>Clean Optic Flow</h1></center>

***

### Clean up displacements traces and make good positions traces

Check the parameters here, they are essential for getting good results

In [ ]:
from face_rhythm.optic_flow import clean_results

## Create position trace from displacements
## This block does a few things:

## 1. Finds outliers: These are currently defined as time points when the integrated position goes beyond some threshold.
##  Note that since displacements are calculated for x and y separately, outlier events are also separated into x outlier events
##  and y outlier events.

## 2. Sets displacements during outlier events to ZERO: There are some parameters below that define the time window (in frames)
##  before and after outliers to also set to zero. Note again, that DISPLACEMENT (the derivative of position) is set to zero, 
##  effectively pausing the position of the ingegrated position.

## 3. Rectifies the position to its 'anchor position': I am defining position as the integrated displacement arising from a STATIC
##  place in the image. Because this analysis is image agnostic, drift naturally occurs. This term counteracts drift by simply
##  relaxing each dot's position back to the center of its displacement analysis window. This term should be as low as possible
##  because it also acts as a high pass filter, thus precluding analysis of slow timescale changes.

## Note that using a standard frequency filter (fir, iir) here for the rectification / relaxation doesn't work well

positions_new_sansOutliers = clean_results.clean_workflow(displacements)

## Make a positions trace where each point centers around it's coordinates instead of zero (used for plotting)
tic = time.time()
positions_new_absolute_sansOutliers = positions_new_sansOutliers + np.squeeze(pointInds_toUse)[:,:,None]
print(f'Made an absolute integrated position. Elapsed time: {round(time.time() - tic , 1)} seconds')

toc_all = time.time() - tic_all
print(f'total elapsed time: {round(toc_all/60,2)} minutes')
print(f'== End outlier removal ==')

In [ ]:
config = helpers.load_config(config_filepath)

save_dir = config['save_dir']

# save_name = f'positions_new_absolute_sansOutliers'
# save_fullPath = f'{save_dir}/{save_name}.npy'
# np.save(save_fullPath , positions_new_absolute_sansOutliers)

save_name = f'positions_new_sansOutliers'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , positions_new_sansOutliers)
config['path_positions'] = save_fullPath

helpers.save_config(config, config_filepath)

In [ ]:
### The upcoming calculations make big data. It's best to delete everything we can from the memory. Consider restarting the kernel and importing just positions_new_sansOutliers
del positions_tracked_outliers
del positions_tracked_outliers_extended
del displacements_simpleOutliersRemoved
del displacements_sansOutliers
del positions_new

# del positions_new_absolute_sansOutliers

del displacements

In [ ]:
gc.collect()

In [ ]:
### Take a quick look at one of the pixel's traces
## Check for significant outliers
%matplotlib notebook

pixelNum_toUse = 300

plt.figure()
plt.plot(positions_new_sansOutliers[pixelNum_toUse,0,:])
# plt.figure()
# plt.plot(displacements[pixelNum_toUse,0,:])


## Convolutional dimensionality reduction
The point here is to do some denoising and to get the number of dots down to a managable number.\
In particular, it is nice for the batched CP decomposition later that the batches can be as big as possible in the temporal dimension, so doing some mild convolutional dim reduction first is helpful.

In [ ]:
## first let's make the convolutional kernel. I like the cosine kernel because it goes to zero.

width_cosKernel = 40

num_dots = pointInds_toUse.shape[0]

def makeDistanceMatrix(n , centerIdx , vid_height , vid_width):
    x,y = np.meshgrid(range(vid_width),range(vid_height)) # note dim 1:X and dim 2:Y
    return np.sqrt((y-int(centerIdx[1]))**2+(x-int(centerIdx[0]))**2)

cosKernel = np.zeros((vid_height , vid_width , num_dots))
cosKernel_mean = np.zeros(num_dots)
for ii in range(num_dots):
    x = makeDistanceMatrix(width_cosKernel , np.squeeze(pointInds_toUse)[ii] , vid_height , vid_width)
    x_norm = x / width_cosKernel
    x_clipped = np.minimum(x_norm , 1)
    cosKernel[:,:,ii] = (np.cos(x_clipped * np.pi) +1) / 2
    tmp = copy.deepcopy(cosKernel[:,:,ii])
    tmp[tmp==0] = np.nan
    cosKernel_mean[ii] = np.nanmean(tmp)

plt.figure()
plt.imshow(cosKernel[:,:,0])

In [ ]:
save_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run7'

save_name = f'cosKernel'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , cosKernel)

In [ ]:
## now the fun part: a bajillion dimensionality reductions

# first, let's make new dots with wider spacing

spacing = 8  ## This is the distance between points in the grid (both in x and y dims)

pts_spaced_convDR = np.ones((np.int64(bbox_subframe_displacement[3] * bbox_subframe_displacement[2] / spacing) ,2)) * np.nan ## preallocation
cc = 0 ## set idx counter
# make spaced out points
for ii in range(len(pts_x_displacement)):
    if (pts_x_displacement[ii]%spacing == 0) and (pts_y_displacement[ii]%spacing == 0):
        pts_spaced_convDR[cc,0] = pts_x_displacement[ii]
        pts_spaced_convDR[cc,1] = pts_y_displacement[ii]
        cc = cc+1

pts_spaced_convDR = np.expand_dims(pts_spaced_convDR,1).astype('single')
pts_spaced_convDR = np.delete(pts_spaced_convDR , np.where(np.isnan(pts_spaced_convDR[:,0,0])) , axis=0)
print(f'number of points: {pts_spaced_convDR.shape[0]}')

In [ ]:
# now let's show the dots we just made

vidNum_toUse = 0 # 0 indexed
frameNum_toUse = 0 # 0 indexed
dot_size = 1

## Define random colors for points in cloud
color_tuples =  list(np.arange(len(pts_x_displacement)))
for ii in range(len(pts_x_displacement)):
    color_tuples[ii] = (np.random.rand(1)[0]*255, np.random.rand(1)[0]*255 , np.random.rand(1)[0]*255)
    
vid = imageio.get_reader(path_vid_allFiles[vidNum_toUse],  'ffmpeg')
frameToSet = 0
frame = vid.get_data(frameNum_toUse) # Get a single frame to use as the first 'previous frame' in calculating optic flow
pointInds_tuple = list(np.arange(pts_spaced_convDR.shape[0])) 
for ii in range(pts_spaced_convDR.shape[0]):
    pointInds_tuple[ii] = tuple(np.squeeze(pts_spaced_convDR[ii,0,:]))
    cv2.circle(frame,pointInds_tuple[ii], dot_size, color_tuples[ii], -1)
cv2.imshow('dots for conv dim red',frame)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
## now let's find coefficients of influence from each original dot onto each new dot

input_traces = np.float32(positions_new_sansOutliers)
num_components = 2
rank_reduced = num_components

dots_old = pointInds_toUse
dots_new = pts_spaced_convDR

pca = sk.decomposition.PCA(n_components=num_components)

positions_convDR_meanSub = np.zeros((dots_new.shape[0] , 2 , input_traces.shape[2]))
output_PCA_loadings = np.zeros((dots_new.shape[0] , 2 , input_traces.shape[2] , num_components))
output_PCA_scores = list(np.zeros(dots_new.shape[0]))
for ii in range(dots_new.shape[0]):
#     print(ii)
    influence_weightings = cosKernel[int(dots_new[ii][0][1]) , int(dots_new[ii][0][0]) , :]
    
    idx_nonZero = np.array(np.where(influence_weightings !=0))[0,:]

    displacements_preConvDR_x = input_traces[idx_nonZero , 0 , :] * influence_weightings[idx_nonZero][:,None]
    displacements_preConvDR_x = displacements_preConvDR_x - np.mean(displacements_preConvDR_x , axis=1)[:,None]
    displacements_preConvDR_y = input_traces[idx_nonZero , 1 , :] * influence_weightings[idx_nonZero][:,None]
    displacements_preConvDR_y = displacements_preConvDR_y - np.mean(displacements_preConvDR_y , axis=1)[:,None]
    pca.fit(displacements_preConvDR_x)
    output_PCA_loadings[ii,0,:,:] = pca.components_.T
    pca.fit(displacements_preConvDR_y)
    output_PCA_loadings[ii,1,:,:] = pca.components_.T
    
    output_PCA_scores[ii] = np.zeros((2,displacements_preConvDR_y.shape[0] , num_components))
    output_PCA_scores[ii][0,:,:] = np.dot( displacements_preConvDR_x  ,  output_PCA_loadings[ii,0,:,:] )
    output_PCA_scores[ii][1,:,:] = np.dot( displacements_preConvDR_y  ,  output_PCA_loadings[ii,1,:,:] )
    positions_convDR_meanSub[ii,0,:] = np.mean(np.dot( output_PCA_loadings[ii,0,:,:rank_reduced] , output_PCA_scores[ii][0,:,:rank_reduced].T ) , axis=1) / cosKernel_mean[ii]
    positions_convDR_meanSub[ii,1,:] = np.mean(np.dot( output_PCA_loadings[ii,1,:,:rank_reduced] , output_PCA_scores[ii][1,:,:rank_reduced].T ) , axis=1) / cosKernel_mean[ii]

In [ ]:
## now let's find coefficients of influence from each original dot onto each new dot
# CAUTION: Huge memory requirement (Lower number of CPUs in the pool to decrease memory usage. For my runs (1 hr at 120hz, downsampling to ~400 dots, I use 18 cores and it
# uses around 150GB of memory. Use single thread version otherwise)

tic = time.time()

input_traces = np.float32(positions_new_sansOutliers)
num_components = 2
rank_reduced = num_components

dots_old = pointInds_toUse
dots_new = pts_spaced_convDR

pca = sk.decomposition.PCA(n_components=num_components)


def makeConvDR(ii):
#     print(ii)
    influence_weightings = cosKernel[int(dots_new[ii][0][1]) , int(dots_new[ii][0][0]) , :]
    
    idx_nonZero = np.array(np.where(influence_weightings !=0))[0,:]

    displacements_preConvDR_x = input_traces[idx_nonZero , 0 , :] * influence_weightings[idx_nonZero][:,None]
    displacements_preConvDR_x = displacements_preConvDR_x - np.mean(displacements_preConvDR_x , axis=1)[:,None]
    displacements_preConvDR_y = input_traces[idx_nonZero , 1 , :] * influence_weightings[idx_nonZero][:,None]
    displacements_preConvDR_y = displacements_preConvDR_y - np.mean(displacements_preConvDR_y , axis=1)[:,None]
    pca.fit(displacements_preConvDR_x)
    output_PCA_loadings0 = pca.components_.T
    pca.fit(displacements_preConvDR_y)
    output_PCA_loadings1 = pca.components_.T
    
    output_PCA_scores0 = np.dot( displacements_preConvDR_x  ,  output_PCA_loadings0 )
    output_PCA_scores1 = np.dot( displacements_preConvDR_y  ,  output_PCA_loadings1 )
    positions_convDR_meanSub = np.zeros((2,input_traces.shape[2]))
    positions_convDR_meanSub[0,:] = np.mean(np.dot( output_PCA_loadings0[:,:rank_reduced] , output_PCA_scores0[:,:rank_reduced].T ) , axis=1) / cosKernel_mean[ii]
    positions_convDR_meanSub[1,:] = np.mean(np.dot( output_PCA_loadings1[:,:rank_reduced] , output_PCA_scores1[:,:rank_reduced].T ) , axis=1) / cosKernel_mean[ii]
    return positions_convDR_meanSub

# p = Pool(multiprocessing.cpu_count())
p = Pool(int(multiprocessing.cpu_count()/3))
positions_convDR_meanSub_list = p.map(makeConvDR , range(dots_new.shape[0]))
p.close()
p.terminate()
p.join()

positions_convDR_meanSub = np.zeros((dots_new.shape[0] , 2 , input_traces.shape[2]))
for ii in range(dots_new.shape[0]):
    positions_convDR_meanSub[ii,:,:] = positions_convDR_meanSub_list[ii]
    
print(f'Time elapsed: {round((time.time() - tic)/60,2)} minutes')

In [ ]:
p.close()
p.terminate()
p.join()

In [ ]:
save_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run7'

save_name = f'positions_convDR_meanSub'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , positions_convDR_meanSub)

save_name = f'pts_spaced_convDR'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , pts_spaced_convDR)

In [ ]:
load_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run6'

load_name = f'positions_convDR_meanSub'
load_fullPath = f'{load_dir}/{load_name}.npy'
positions_convDR_meanSub = np.load(load_fullPath)

load_name = f'pts_spaced_convDR'
load_fullPath = f'{load_dir}/{load_name}.npy'
pts_spaced_convDR = np.load(load_fullPath)


positions_convDR_absolute = (positions_convDR_meanSub + np.squeeze(pts_spaced_convDR)[:,:,None])

In [ ]:
### Display video of displacement dot-grid after outlier removal
## Just for observation purposes

# positions_toUse = positions_new_absolute_sansOutliers
positions_toUse = (positions_convDR_meanSub + np.squeeze(pts_spaced_convDR)[:,:,None])

# vidNums_toUse = range(numVids) ## note zero indexing!
vidNums_toUse = range(3) ## note zero indexing!

if type(vidNums_toUse) == int:
    vidNums_toUse = np.array([vidNums_toUse])

dot_size = 1

printFPS_pref = 0
fps_counterPeriod = 10 ## number of frames to do a tic toc over


## Define random colors for points in cloud
color_tuples =  list(np.arange(positions_toUse.shape[0]))
for ii in range(positions_toUse.shape[0]):
    color_tuples[ii] = (np.random.rand(1)[0]*255, np.random.rand(1)[0]*255 , np.random.rand(1)[0]*255)
#     color_tuples[ii] = (0,255,255)
        
        

## Main loop to pull out displacements in each video   
ind_concat = int(np.hstack([0 , np.cumsum(numFrames_allFiles)])[vidNums_toUse[0]])

fps = 0
tic_fps = time.time()
for iter_vid , vidNum_iter in enumerate(vidNums_toUse):
    path_vid = path_vid_allFiles[vidNum_iter]
    vid = imageio.get_reader(path_vid,  'ffmpeg')

    video = cv2.VideoCapture(path_vid)
    numFrames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    for iter_frame , new_frame in enumerate(vid):
        for ii in range(positions_toUse.shape[0]):
            pointInds_tracked_tuple = tuple(np.int64(np.squeeze(positions_toUse[ii,:,ind_concat])))
            cv2.circle(new_frame,pointInds_tracked_tuple, dot_size, color_tuples[ii], -1)
        
        cv2.putText(new_frame, f'frame #: {iter_frame}/{numFrames}-ish', org=(10,20), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
        cv2.putText(new_frame, f'vid #: {iter_vid+1}/{len(vidNums_toUse)}', org=(10,40), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
        cv2.putText(new_frame, f'total frame #: {ind_concat+1}/{numFrames_total_rough}-ish', org=(10,60), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
        cv2.putText(new_frame, f'fps: {np.uint32(fps)}', org=(10,80), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
        cv2.imshow('post outlier removal',new_frame)


        k = cv2.waitKey(1) & 0xff
        if k == 27 : break
            
        ind_concat = ind_concat+1

        
        if ind_concat%fps_counterPeriod==0:
            elapsed = time.time() - tic_fps
            fps = fps_counterPeriod/elapsed
            if printFPS_pref:
                print(fps)
            tic_fps = time.time()
            

cv2.destroyAllWindows()


In [ ]:
cv2.destroyAllWindows()

***
<center><h1>Analysis</h1></center>

***

### Decompose and Analyze the Data in different ways

Below you'll find the following:
- PCA done on raw positions
- Spectral analysis of every pixels to transoform the basis to be oscillatory
- TCA done on the spectra
- A lonely t-SNE plot of the temporal factors

### PCA
The X and Y displacements are concatenated and run together. Something interesting to try would be to transform to polar coordinates, concatenate and run that way. Maybe TCA on the positions with magnitude vs angle being one of the dimensions would make sense.

In [ ]:
%matplotlib notebook
# input_dimRed = np.squeeze(positions_new_sansOutliers[:,1,:])
tmp_x = np.squeeze(positions_convDR_meanSub[:,0,:])
tmp_y = np.squeeze(positions_convDR_meanSub[:,1,:])
                   
input_dimRed_meanSub = np.concatenate((tmp_x - tmp_x.mean(1)[:,None] , tmp_y - tmp_y.mean(1)[:,None]) , axis=1 )
# input_dimRed_concat = np.concatenate( (np.squeeze(positions_new_sansOutliers[:,0,:]) , np.squeeze(positions_new_sansOutliers[:,1,:])) , axis=1)

# input_dimRed_meanSub = input_dimRed_concat - np.matlib.repmat( np.expand_dims(np.mean(input_dimRed_concat , axis=1) , axis=1) , 1 , input_dimRed_concat.shape[1])
# input_dimRed_meanSub = input_dimRed_concat - input_dimRed_concat.mean(1)[:,None]

pca = sk.decomposition.PCA(n_components=10)
# pca = sk.decomposition.FastICA(n_components=10)
pca.fit(np.float32(input_dimRed_meanSub))
output_PCA = pca.components_.transpose()
scores_points = np.dot(input_dimRed_meanSub , output_PCA)

# plt.figure()
# plt.imshow(positions_tracked[:,])
plt.figure()
plt.plot(output_PCA[:,:3])
plt.figure()
plt.plot(pca.explained_variance_)
plt.figure()
plt.plot(output_PCA[:,0] , output_PCA[:,1]  , linewidth=.1)

plt.figure()
plt.plot(scores_points[:,:3])

In [ ]:
# del input_dimRed_concat
del input_dimRed_meanSub
del tmp_x
del tmp_y
del output_PCA
gc.collect()

### Positional TCA

In [ ]:
### Prepare Tensorly

pref_useGPU = 0


tl.set_backend('pytorch')

# If the input is small ( < half the size of your GPU memory) and you have CUDA, set to 'gpu'. It's super fast.
if pref_useGPU:
    cuda_device_number = torch.cuda.current_device()
    print(f"using CUDA device: 'cuda:{cuda_device_number}'")
    device = f'cuda:{cuda_device_number}'
else:
    print(f"using CPU")
    device = 'cpu'  


## Prepare the input tensor
print(f'== Starting loading tensor ==')
tic = time.time()

input_tensor = tl.tensor(positions_convDR_meanSub, dtype=tl.float32, device=device, requires_grad=False)
print(f'== Finished loading tensor. Elapsed time: {round(time.time() - tic,2)} seconds ==')

print(f'Size of input (spectrogram): {input_tensor.shape}')

print(f'{round(sys.getsizeof(input_dimRed_meanSub)/1000000000,3)} GB')

In [ ]:
### Fit TCA model
## If the input is small, set init='svd'

rank = 4

weights, factors_positional = tensorly.decomposition.parafac(input_tensor, init='random', tol=1e-06, n_iter_max=800, rank=rank, verbose=True, orthogonalise=False, random_state=1234)
# weights, factors = tensorly.decomposition.non_negative_parafac(Sxx_allPixels_tensor[:,:,:,:], init='svd', tol=1e-05, n_iter_max=100, rank=rank, verbose=True,)
# weights, factors = tensorly.decomposition.parafac(Sxx_allPixels_tensor, init='random', tol=1e-05, n_iter_max=1000, rank=rank, verbose=True)

In [ ]:
## make numpy version of tensorly output

factors_toUse = factors_positional


if pref_useGPU:
    factors_np_positional = list(np.arange(len(factors_toUse)))
    for ii in range(len(factors_toUse)):
#         factors_np[ii] = tl.tensor(factors[ii] , dtype=tl.float32 , device='cpu')
        factors_np_positional[ii] = factors_toUse[ii].cpu().clone().detach().numpy()
else:
    factors_np_positional = []
    for ii in range(len(factors_toUse)):
        factors_np_positional.append(np.array(factors_toUse[ii]))

In [ ]:
len(factors_np_positional)
input_tensor.shape

In [ ]:
%matplotlib notebook

factors_toUse = factors_np_positional
modelRank = factors_toUse[0].shape[1]
## just for plotting in case 
if 'Fs' not in globals():
    Fs = 120

plt.figure()
# plt.plot(np.arange(factors_toUse.factors(4)[0][2].shape[0])/Fs , factors_toUse.factors(4)[0][2])
factors_temporal = scipy.stats.zscore(factors_toUse[2][:,:] , axis=0)
factors_temporal = factors_toUse[2][:,:]
# factors_temporal = scipy.stats.zscore(factors_temporal_reconstructed , axis=0)
# plt.plot(np.arange(factors_temporal.shape[0])/Fs, factors_temporal[:,:])
plt.plot(np.arange(factors_temporal.shape[0])/Fs, factors_temporal[:,])
# plt.plot(factors_temporal[:,:])
plt.legend(np.arange(modelRank)+1)
plt.xlabel('time (s)')
plt.ylabel('a.u.')


plt.figure()
plt.plot(factors_toUse[1][:,:])
plt.legend(np.arange(modelRank)+1)
plt.xlabel('x vs. y')
plt.ylabel('a.u.')

plt.figure()
plt.plot(factors_toUse[0][:,:])
plt.legend(np.arange(modelRank)+1)
plt.xlabel('pixel number')
plt.ylabel('a.u.')


plt.figure()
plt.imshow(np.single(np.corrcoef(factors_toUse[2][:,:].T)))

# input_dimRed = factors_toUse[2][:,:]
# # input_dimRed_meanSub = 
# pca = sk.decomposition.PCA(n_components=modelRank-2)
# # pca = sk.decomposition.FactorAnalysis(n_components=3)
# pca.fit(np.single(input_dimRed).transpose())
# output_PCA = pca.components_.transpose()
# # scores_points = np.dot(ensemble.factors(4)[0][2] , output_PCA)

# plt.figure()
# plt.plot(output_PCA)

In [ ]:
# Display video of factors

factors_toShow = np.arange(factors_np_positional[0].shape[1])  # zero-indexed
# factors_toShow = [3]  # zero-indexed

for factor_iter in factors_toShow:

    # vidNums_toUse = range(numVids) ## note zero indexing!
    vidNums_toUse = 0 ## note zero indexing!

    if type(vidNums_toUse) == int:
        vidNums_toUse = np.array([vidNums_toUse])

    dot_size = 2

    printFPS_pref = 0
    fps_counterPeriod = 10 ## number of frames to do a tic toc over

#     modelRank_toUse = 5
    factor_toShow = factor_iter+1
    save_pref= 0

    # save_dir = "F:\\RH_Local\\Rich data\\camera data"
    save_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run7'
    save_fileName = f'factor {factor_toShow}'
    # save_pathFull = f'{save_dir}\\{save_fileName}.avi'
    save_pathFull = f'{save_dir}/{save_fileName}.avi'

    # ensemble_toUse = ensemble
    ensemble_toUse = factors_np_positional
    positions_toUse = positions_convDR_absolute

    factor_toShow = factor_toShow-1
    # input_scores = ensemble_toUse.factors(modelRank_toUse)[0][0]
    input_scores = np.single(ensemble_toUse[0])

    range_toUse = np.ceil(np.max(input_scores[:,factor_toShow]) - np.min(input_scores[:,factor_toShow])) + 1
    offset_toUse = np.min(input_scores[:,factor_toShow])
    scores_norm = input_scores[:,factor_toShow] - offset_toUse
    scores_norm = (scores_norm / np.max(scores_norm)) *1000
    cmap = matplotlib.cm.get_cmap('hot', 1000)
    # cmap_viridis(np.arange(range_toUse))

    colormap_tuples =  list(np.arange(positions_toUse.shape[0]))
    for ii in range(positions_toUse.shape[0]):
        colormap_tuples[ii] = list(np.flip((np.array(cmap(np.int64(scores_norm[ii]))) *255)[:3]))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    if save_pref:
        print(f'saving to file {save_pathFull}')
        out = cv2.VideoWriter(save_pathFull, fourcc, Fs, (np.int64(vid_width), np.int64(vid_height)))


    ## Main loop to pull out displacements in each video   
    ind_concat = int(np.hstack([0 , np.cumsum(numFrames_allFiles)])[vidNums_toUse[0]])

    fps = 0
    tic_fps = time.time()
    for iter_vid , vidNum_iter in enumerate(vidNums_toUse):
        path_vid = path_vid_allFiles[vidNum_iter]
        vid = imageio.get_reader(path_vid,  'ffmpeg')

#         numFrames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        numFrames = 1000

#         frameToSet = 0
#         video.set(1,frameToSet)

        for iter_frame , new_frame in enumerate(vid):

#             ind_currentVid = np.int64(video.get(cv2.CAP_PROP_POS_FRAMES))
            if iter_frame >= numFrames:
                break
#             ok, new_frame = video.read()

            for ii in range(positions_toUse.shape[0]):
                pointInds_tracked_tuple = tuple(np.int64(np.squeeze(positions_toUse[ii,:,ind_concat])))
                cv2.circle(new_frame,pointInds_tracked_tuple, dot_size, colormap_tuples[ii], -1)
            if save_pref:
                out.write(new_frame)

#             Sxx_frameNum = round( ind_currentVid / (positions_toUse.shape[2] / Sxx_allPixels.shape[2]) ,1)
            cv2.putText(new_frame, f'frame #: {iter_frame}/{numFrames}', org=(10,20), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
#             cv2.putText(new_frame, f'frame #: {Sxx_frameNum}', org=(10,20), fontFace=1, fontScale=1, color=(255,255,255), thickness=2)
#             cv2.putText(new_frame, f'vid #: {iter+1}/{len(vidNums_toUse)}', org=(10,40), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
            cv2.putText(new_frame, f'total frame #: {ind_concat+1}/{positions_toUse.shape[2]}', org=(10,60), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
            cv2.putText(new_frame, f'fps: {np.uint32(fps)}', org=(10,80), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
            cv2.putText(new_frame, f'factor num: {factor_iter+1} / {np.max(factors_toShow)+1}', org=(10,100), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
            cv2.imshow('test',new_frame)


            k = cv2.waitKey(1) & 0xff
            if k == 27 : break

            ind_concat = ind_concat+1


            if ind_concat%fps_counterPeriod==0:
                elapsed = time.time() - tic_fps
                fps = fps_counterPeriod/elapsed
                if printFPS_pref:
                    print(fps)
                tic_fps = time.time()


out.release()
video.release()
cv2.destroyAllWindows()

In [ ]:
# Display Factors

positions_toUse = positions_convDR_absolute

# vidNums_toUse = range(numVids) ## note zero indexing!
vidNums_toUse = 0 ## note zero indexing!

if type(vidNums_toUse) == int:
    vidNums_toUse = np.array([vidNums_toUse])

dot_size = 2

printFPS_pref = 0
fps_counterPeriod = 10 ## number of frames to do a tic toc over

PC_toShow = 3
PC_toShow = PC_toShow-1

range_toUse = np.ceil(np.max(scores_points[:,PC_toShow]) - np.min(scores_points[:,PC_toShow])) + 1
offset_toUse = np.min(scores_points[:,PC_toShow])
scores_norm = scores_points[:,PC_toShow] - offset_toUse
scores_norm = (scores_norm / np.max(scores_norm)) *1000
cmap = matplotlib.cm.get_cmap('hot', 1000)


colormap_tuples =  list(np.arange(pts_spaced_convDR.shape[0]))
for ii in range(pts_spaced_convDR.shape[0]):
    colormap_tuples[ii] = list(np.flip((np.array(cmap(np.int64(scores_norm[ii]))) *255)[:3]))
        
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
save_pref = 0
if save_pref:
    out = cv2.VideoWriter('F:\RH_Local\Rich data\camera data\output.avi',fourcc, Fs, (vid_width, vid_height))
      

## Main loop to pull out displacements in each video   
ind_concat = int(np.hstack([0 , np.cumsum(numFrames_allFiles)])[vidNums_toUse[0]])

fps = 0
tic_fps = time.time()
for iter_vid , vidNum_iter in enumerate(vidNums_toUse):
    path_vid = path_vid_allFiles[vidNum_iter]
    vid = imageio.get_reader(path_vid,  'ffmpeg')
        
    numFrames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    for iter_frame , new_frame in enumerate(vid):
        
        for ii in range(positions_toUse.shape[0]):
            pointInds_tracked_tuple = tuple(np.int64(np.squeeze(positions_toUse[ii,:,ind_concat])))
            cv2.circle(new_frame,pointInds_tracked_tuple, dot_size, colormap_tuples[ii], -1)
        if save_pref:
            out.write(new_frame)
            
        cv2.putText(new_frame, f'frame #: {iter_frame}/{numFrames}', org=(10,20), fontFace=1, fontScale=1, color=(255,255,255), thickness=2)
        cv2.putText(new_frame, f'vid #: {iter_vid+1}/{len(vidNums_toUse)}', org=(10,40), fontFace=1, fontScale=1, color=(255,255,255), thickness=2)
#         cv2.putText(new_frame, f'total frame #: {ind_concat+1}/{numFrames_total}', org=(10,60), fontFace=1, fontScale=1, color=(255,255,255), thickness=2)
        cv2.putText(new_frame, f'fps: {np.uint32(fps)}', org=(10,80), fontFace=1, fontScale=1, color=(255,255,255), thickness=2)
        cv2.putText(new_frame, f'PC #: {PC_toShow+1}', org=(10,100), fontFace=1, fontScale=1, color=(255,255,255), thickness=2)
        cv2.imshow('PCs',new_frame)


        k = cv2.waitKey(1) & 0xff
        if k == 27 : break
            
        ind_concat = ind_concat+1

        
        if ind_concat%fps_counterPeriod==0:
            elapsed = time.time() - tic_fps
            fps = fps_counterPeriod/elapsed
            if printFPS_pref:
                print(fps)
            tic_fps = time.time()
            

out.release()
video.release()
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

### Spectral Analysis
I've played with a few different methods. While multiresolution methods seems ideal for this use-case, It just ends up severly overrepresenting low frequency factors, making noisier high frequency factors, and doing an overall worse job at reconstruction.
A good ol' multitaper short time fourier transform seems to work fine. Adding in raw positions to subsequent dimensionality reduction later on seems like a natural thing to do, as single resolution spectral analysis ends up kind of ignoring slower dynamics.

In [ ]:
load_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run3'

load_name = f'settings_cqt'
load_fullPath = f'{load_dir}/{load_name}.npy'
settings_cqt = np.load(load_fullPath , allow_pickle=True)[()]


hop_length = settings_cqt['hop_length']
fmin = settings_cqt['fmin']
sr = settings_cqt['sr']
bins_per_octave = settings_cqt['bins_per_octave']
n_bins = settings_cqt['n_bins']
print(settings_cqt)

In [ ]:
eps = 1.19209e-07 # float32 epsilon

hop_length = 16
fmin_rough = 2
sr = Fs
n_bins = 50

bins_per_octave = int(np.round((n_bins) / np.log2( (Fs/2)/fmin_rough )))
fmin = ( (Fs/2)/(2**((n_bins)/bins_per_octave)) ) - (2*eps)
fmax = fmin*(2**((n_bins)/bins_per_octave))

freqs_Sxx = fmin*(2**((np.arange(n_bins)+1)/bins_per_octave))

print(f'bins_per_octave: {round(bins_per_octave)} bins/octave')
print(f'minimum frequency (fmin): {round(fmin,3)} Hz')
print(f'maximum frequency (fmax): {round(fmax,8)} Hz')
print(f'Nyquist                 : {sr/2} Hz')
print(f'number of frequencies:    {n_bins} bins')
plt.figure()
plt.plot(freqs_Sxx)
print(f'Frequencies: {np.round(freqs_Sxx , 3)}')

In [ ]:
### Parameters for multitaper short-time Fourier transform

settings_cqt =	{
    "hop_length": hop_length,  # number of samples between successive temporal samples
    "sr": sr,  # sampling rate (Fs)
    "n_bins": n_bins,  # Number of frequency bins, starting at fmin
    "bins_per_octave": bins_per_octave,  #     Number of bins per octave
    "fmin": fmin,  # Minimum frequency
    "fmax": fmax,  # Maximum frequency (just for reference, not input to librosa.cqt)
    }

In [ ]:
save_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run7'

save_name = f'settings_cqt'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , settings_cqt)


save_name = f'freqs_Sxx'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , freqs_Sxx)

In [ ]:
### CQT spectrogram for every pixel
## this code was previously parallelized, but it's pretty quick compared to the other steps, so might as well keep it simple

print(f'== Starting CQT spectrogram calculations ==')
tic_all = time.time()

## define positions traces to use
# input_sgram = np.single(np.squeeze(positions_new_sansOutliers))[:,:,:]
input_sgram = np.single(np.squeeze(positions_convDR_meanSub))[:,:,:]

## get parameters
hop_length = hop_length
fmin_rough = fmin_rough
sr = sr
n_bins = n_bins
bins_per_octave = bins_per_octave
fmin = fmin

## make a single spectrogram to get some size parameters for preallocation
Sxx = librosa.cqt(np.squeeze(input_sgram[0,0,:]), 
                  sr=sr, 
                  hop_length=hop_length, 
                  fmin=fmin, 
                  n_bins=n_bins, 
                  bins_per_octave=bins_per_octave, 
                  window='hann')

# preallocation
tic = time.time()
print(f'preallocating')
Sxx_allPixels = np.single(np.zeros((input_sgram.shape[0] , Sxx.shape[0] , Sxx.shape[1] , 2)))  
print(f'preallocation done. Elapsed time: {round((time.time() - tic) , 2)} seconds')

print(f'starting spectrogram calculation')
for ii in range(input_sgram.shape[0]):
    ## progress tracking
    if ii%50 ==0:
        print(f'{ii} / {Sxx_allPixels.shape[0]}')
    elif ii==1:
        print(f'{ii} / {Sxx_allPixels.shape[0]}')
    
    ## iterated over x and y
    for jj in range(2):
        tmp_input_sgram = np.squeeze(input_sgram[ii,jj,:])


        tmp = librosa.cqt(np.squeeze(input_sgram[ii,jj,:]), 
                          sr=sr, 
                          hop_length=hop_length, 
                          fmin=fmin, 
                          n_bins=n_bins, 
                          bins_per_octave=bins_per_octave, 
                          window='hann')
        
        ## normalization
        tmp = abs(tmp) * freqs_Sxx[:,None]
#         tmp = scipy.stats.zscore(tmp , axis=0)
#         tmp = test - np.min(tmp , axis=0)[None,:]
#         tmp = scipy.stats.zscore(tmp , axis=1)
#         tmp = tmp - np.min(tmp , axis=1)[:,None]

        Sxx_allPixels[ii,:,:,jj] = tmp
# Sxx_allPixels = Sxx_allPixels / np.std(Sxx_allPixels , axis=1)[:,None,:,:]

print(f'completed spectrogram calculation')
print('Info about Sxx_allPixels:\n')
print(f'Shape: {Sxx_allPixels.shape}')
print(f'Number of elements: {Sxx_allPixels.shape[0]*Sxx_allPixels.shape[1]*Sxx_allPixels.shape[2]*Sxx_allPixels.shape[3]}')
print(f'Data type: {Sxx_allPixels.dtype}')
print(f'size of Sxx_allPixels: {round(sys.getsizeof(Sxx_allPixels)/1000000000,3)} GB')
print(f'== Spectrograms computed. Total elapsed time: {round((time.time() - tic_all)/60 , 2)} minutes ==')
      

In [ ]:
### Normalize the spectrograms so that each time point has a similar cumulative spectral amplitude across all dots (basically, sum of power of all frequencies from all dots at a particular time should equal one)
## hold onto the normFactor variable because you can use to it to undo the normalization after subsequent steps
Sxx_allPixels_normFactor = np.mean(np.sum(Sxx_allPixels , axis=1) , axis=0)
Sxx_allPixels_norm = Sxx_allPixels / Sxx_allPixels_normFactor[None,None,:,:]
Sxx_allPixels_norm.shape

In [ ]:
plt.figure()
plt.imshow(Sxx_allPixels_norm[500,:,:,0]  , aspect='auto' , cmap='hot' , origin='lower')

plt.figure()
plt.plot(Sxx_allPixels_normFactor)

In [ ]:
Sxx_positional.shape

In [ ]:
## define positions traces to use
input_sgram = np.single(np.squeeze(factors_np_positional[2][:,3]))

## get parameters
hop_length = 8
fmin_rough = fmin_rough
sr = sr
n_bins = n_bins
bins_per_octave = bins_per_octave
fmin = fmin

## make a single spectrogram to get some size parameters for preallocation
Sxx_positional = librosa.cqt(np.squeeze(input_sgram), 
                            sr=sr, 
                            hop_length=hop_length, 
                            fmin=fmin, 
                            n_bins=n_bins, 
                            bins_per_octave=bins_per_octave, 
                            window=('hann'),
                            filter_scale=0.8)
Sxx_positional = abs(Sxx_positional) * freqs_Sxx[:,None]
# Sxx_positional = abs(Sxx_positional)
# test = scipy.stats.zscore(Sxx_positional , axis=0)
test_std = np.std(Sxx_positional , axis=0)
test_sum = np.sum(Sxx_positional , axis=0)
test = Sxx_positional / (test_std[None,:] )
# test = (Sxx_positional) / (test_sum[None,:])
# test = test - np.min(test , axis=0)[None,:]

test2 = scipy.stats.zscore(Sxx_positional , axis=1)
test2 = test2 - np.min(test2 , axis=1)[:,None]

plt.figure()
plt.imshow(Sxx_positional , aspect='auto' , cmap='hot' , origin='lower')
plt.figure()
plt.imshow(test , aspect='auto' , cmap='hot' , origin='lower')
plt.figure()
plt.imshow(test2 , aspect='auto' , cmap='hot' , origin='lower')


### Saving & Loading before TCA

In [ ]:
save_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run7'

save_name = f'Sxx_allPixels'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , Sxx_allPixels)

save_name = f'Sxx_allPixels_norm'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , Sxx_allPixels_norm)

save_name = f'Sxx_allPixels_normFactor'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , Sxx_allPixels_normFactor)


save_name = f'freqs_Sxx'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , freqs_Sxx)

In [ ]:
load_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run2'

load_name = f'freqs_Sxx'
load_fullPath = f'{load_dir}/{load_name}.npy'
freqs_Sxx = np.load(load_fullPath)

load_name = f'Sxx_allPixels'
load_fullPath = f'{load_dir}/{load_name}.npy'
Sxx_allPixels = np.load(load_fullPath)

load_name = f'Fs'
load_fullPath = f'{load_dir}/{load_name}.npy'
Fs = np.load(load_fullPath)

### TCA
There are two major tensor packages, one is tensortools (made by an acquaintance named Alex Williams) and the other is Tensorly.
Tensorly seems to be more packaged up and has some options to use some advanced backends like torch, tf, and mxnet. Though there are
a couple of nice features in tensortools that Tensorly doesn't have, though. Generally tensortools gives better reconstructions, but takes
much longer to do it.

In [ ]:
### Prepare Tensorly

pref_useGPU = 1


tl.set_backend('pytorch')

# If the input is small ( < half the size of your GPU memory) and you have CUDA, set to 'gpu'. It's super fast.
if pref_useGPU:
    cuda_device_number = torch.cuda.current_device()
    print(f"using CUDA device: 'cuda:{cuda_device_number}'")
    device = f'cuda:{cuda_device_number}'
else:
    print(f"using CPU")
    device = 'cpu'  


## Prepare the input tensor
print(f'== Starting loading tensor ==')
tic = time.time()
Sxx_allPixels_tensor = tl.tensor(tmp[:,:,:,:], dtype=tl.float32, device=device, requires_grad=False)
print(f'== Finished loading tensor. Elapsed time: {round(time.time() - tic,2)} seconds ==')

print(f'Size of input (spectrogram): {Sxx_allPixels_tensor.shape}')

print(f'{round(sys.getsizeof(Sxx_allPixels)/1000000000,3)} GB')

In [ ]:
del Sxx_allPixels
gc.collect()

In [ ]:
### Fit TCA model
## If the input is small, set init='svd'

rank = 8

weights, factors = tensorly.decomposition.non_negative_parafac(Sxx_allPixels_tensor[:,:,:,:], init='random', tol=1e-07, n_iter_max=400, rank=rank, verbose=True, orthogonalise=False, random_state=1234)
# weights, factors = tensorly.decomposition.non_negative_parafac(Sxx_allPixels_tensor[:,:,:,:], init='svd', tol=1e-05, n_iter_max=100, rank=rank, verbose=True,)
# weights, factors = tensorly.decomposition.parafac(Sxx_allPixels_tensor, init='random', tol=1e-05, n_iter_max=1000, rank=rank, verbose=True)

In [ ]:
## make numpy version of tensorly output

factors_toUse = factors


if pref_useGPU:
    factors_np = list(np.arange(len(factors)))
    for ii in range(len(factors)):
#         factors_np[ii] = tl.tensor(factors[ii] , dtype=tl.float32 , device='cpu')
        factors_np[ii] = factors[ii].cpu().clone().detach().numpy()
else:
    factors_np = []
    for ii in range(len(factors_toUse)):
        factors_np.append(np.array(factors_toUse[ii]))

In [ ]:
save_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run7'

save_name = f'factors_np'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , factors_np)

In [ ]:
load_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run6'

load_name = f'factors_np_batchCPsqrt'
load_fullPath = f'{load_dir}/{load_name}.npy'
factors_np = np.load(load_fullPath , allow_pickle=True)

load_name = f'freqs_Sxx'
load_fullPath = f'{load_dir}/{load_name}.npy'

freqs_Sxx = np.load(load_fullPath)

In [ ]:
%matplotlib notebook

factors_toUse = factors_np
modelRank = factors_toUse[0].shape[1]
## just for plotting in case 
if 'Fs' not in globals():
    Fs = 120

plt.figure()
# plt.plot(np.arange(factors_toUse.factors(4)[0][2].shape[0])/Fs , factors_toUse.factors(4)[0][2])
# factors_temporal = scipy.stats.zscore(factors_toUse[2][:,:] , axis=0)
factors_temporal = factors_toUse[2][:,:]
# factors_temporal = scipy.stats.zscore(factors_temporal_reconstructed , axis=0)
# plt.plot(np.arange(factors_temporal.shape[0])/Fs, factors_temporal[:,:])
plt.plot(np.arange(factors_temporal.shape[0])/Fs, factors_temporal[:,])
# plt.plot(factors_temporal[:,:])
plt.legend(np.arange(modelRank)+1)
plt.xlabel('time (s)')
plt.ylabel('a.u.')

plt.figure()
# plt.plot(np.arange(factors_toUse.factors(4)[0][2].shape[0])/Fs , factors_toUse.factors(4)[0][2])
# factors_temporal = scipy.stats.zscore(factors_toUse[2][:,:] , axis=0)
factors_temporal = factors_toUse[2][:,:] * (np.mean(Sxx_allPixels_normFactor , axis=1)[:,None] **(2/5))
# factors_temporal = scipy.stats.zscore(factors_temporal_reconstructed , axis=0)
# plt.plot(np.arange(factors_temporal.shape[0])/Fs, factors_temporal[:,:])
plt.plot(np.arange(factors_temporal.shape[0])/Fs, factors_temporal[:,:])
# plt.plot(factors_temporal[:,:])
plt.legend(np.arange(modelRank)+1)
plt.xlabel('time (s)')
plt.ylabel('a.u.')

plt.figure()
plt.plot(freqs_Sxx , (factors_toUse[1][:,:]))
# plt.plot(freqXaxis , (factors_toUse[1][:,:]))
# plt.plot(f , (factors_toUse[1][:,:]))
# plt.plot((factors_toUse[1][:,:]))
plt.legend(np.arange(modelRank)+1)
plt.xscale('log')
plt.xlabel('frequency (Hz)')
plt.ylabel('a.u.')
# plt.xscale('log')

plt.figure()
plt.plot(factors_toUse[3][:,:])
plt.legend(np.arange(modelRank)+1)
plt.xlabel('x vs. y')
plt.ylabel('a.u.')

plt.figure()
plt.plot(factors_toUse[0][:,:])
plt.legend(np.arange(modelRank)+1)
plt.xlabel('pixel number')
plt.ylabel('a.u.')


plt.figure()
plt.imshow(np.single(np.corrcoef(factors_toUse[2][:,:].T)))

input_dimRed = factors_toUse[2][:,:]
# input_dimRed_meanSub = 
pca = sk.decomposition.PCA(n_components=modelRank-2)
# pca = sk.decomposition.FactorAnalysis(n_components=3)
pca.fit(np.single(input_dimRed).transpose())
output_PCA = pca.components_.transpose()
# scores_points = np.dot(ensemble.factors(4)[0][2] , output_PCA)

plt.figure()
plt.plot(output_PCA[:,5])

In [ ]:
## Do some cross-correlations
input_factors = factors_np
factors_xcorr = np.zeros((input_factors[2].shape[0] , input_factors[2].shape[1] , input_factors[2].shape[1]))
for ii in range(input_factors[2].shape[1]):
    for jj in range(input_factors[2].shape[1]):
        factors_xcorr[:,ii,jj] = scipy.signal.correlate(input_factors[2][:,ii] , input_factors[2][:,jj] , mode='same')
    print(ii)


for ii in range(factors_xcorr.shape[1]):
    print(ii+1)
    plt.figure()
    plt.plot(np.squeeze(factors_xcorr[:,ii,:]))
    plt.legend(np.arange(modelRank)+1)


In [ ]:
load_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run7'
load_name = f'positions_convDR_meanSub'
load_fullPath = f'{load_dir}/{load_name}.npy'

positions_convDR_meanSub = np.load(load_fullPath)

In [ ]:
positions_convDR_absolute = positions_convDR_meanSub + np.squeeze(pts_spaced_convDR)[:,:,None]

In [ ]:
# Display video of factors

factors_toShow = np.arange(factors_np[0].shape[1])  # zero-indexed
# factors_toShow = [3]  # zero-indexed

for factor_iter in factors_toShow:

    # vidNums_toUse = range(numVids) ## note zero indexing!
    vidNums_toUse = 0 ## note zero indexing!

    if type(vidNums_toUse) == int:
        vidNums_toUse = np.array([vidNums_toUse])

    dot_size = 2

    printFPS_pref = 0
    fps_counterPeriod = 10 ## number of frames to do a tic toc over

#     modelRank_toUse = 5
    factor_toShow = factor_iter+1
    save_pref= 0

    # save_dir = "F:\\RH_Local\\Rich data\\camera data"
    save_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run7'
    save_fileName = f'factor {factor_toShow}'
    # save_pathFull = f'{save_dir}\\{save_fileName}.avi'
    save_pathFull = f'{save_dir}/{save_fileName}.avi'

    # ensemble_toUse = ensemble
    ensemble_toUse = factors_np
    positions_toUse = positions_convDR_absolute

    factor_toShow = factor_toShow-1
    # input_scores = ensemble_toUse.factors(modelRank_toUse)[0][0]
    input_scores = np.single(ensemble_toUse[0])

    range_toUse = np.ceil(np.max(input_scores[:,factor_toShow]) - np.min(input_scores[:,factor_toShow])) + 1
    offset_toUse = np.min(input_scores[:,factor_toShow])
    scores_norm = input_scores[:,factor_toShow] - offset_toUse
    scores_norm = (scores_norm / np.max(scores_norm)) *1000
    cmap = matplotlib.cm.get_cmap('hot', 1000)
    # cmap_viridis(np.arange(range_toUse))

    colormap_tuples =  list(np.arange(positions_toUse.shape[0]))
    for ii in range(positions_toUse.shape[0]):
        colormap_tuples[ii] = list(np.flip((np.array(cmap(np.int64(scores_norm[ii]))) *255)[:3]))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    if save_pref:
        print(f'saving to file {save_pathFull}')
        out = cv2.VideoWriter(save_pathFull, fourcc, Fs, (np.int64(vid_width), np.int64(vid_height)))


    ## Main loop to pull out displacements in each video   
    ind_concat = int(np.hstack([0 , np.cumsum(numFrames_allFiles)])[vidNums_toUse[0]])

    fps = 0
    tic_fps = time.time()
    for iter_vid , vidNum_iter in enumerate(vidNums_toUse):
        path_vid = path_vid_allFiles[vidNum_iter]
        vid = imageio.get_reader(path_vid,  'ffmpeg')

#         numFrames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        numFrames = 1000

#         frameToSet = 0
#         video.set(1,frameToSet)

        for iter_frame , new_frame in enumerate(vid):

#             ind_currentVid = np.int64(video.get(cv2.CAP_PROP_POS_FRAMES))
            if iter_frame >= numFrames:
                break
#             ok, new_frame = video.read()

            for ii in range(positions_toUse.shape[0]):
                pointInds_tracked_tuple = tuple(np.int64(np.squeeze(positions_toUse[ii,:,ind_concat])))
                cv2.circle(new_frame,pointInds_tracked_tuple, dot_size, colormap_tuples[ii], -1)
            if save_pref:
                out.write(new_frame)

#             Sxx_frameNum = round( ind_currentVid / (positions_toUse.shape[2] / Sxx_allPixels.shape[2]) ,1)
            cv2.putText(new_frame, f'frame #: {iter_frame}/{numFrames}', org=(10,20), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
#             cv2.putText(new_frame, f'frame #: {Sxx_frameNum}', org=(10,20), fontFace=1, fontScale=1, color=(255,255,255), thickness=2)
#             cv2.putText(new_frame, f'vid #: {iter+1}/{len(vidNums_toUse)}', org=(10,40), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
            cv2.putText(new_frame, f'total frame #: {ind_concat+1}/{positions_toUse.shape[2]}', org=(10,60), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
            cv2.putText(new_frame, f'fps: {np.uint32(fps)}', org=(10,80), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
            cv2.putText(new_frame, f'factor num: {factor_iter+1} / {np.max(factors_toShow)+1}', org=(10,100), fontFace=1, fontScale=1, color=(255,255,255), thickness=1)
            cv2.imshow('test',new_frame)


            k = cv2.waitKey(1) & 0xff
            if k == 27 : break

            ind_concat = ind_concat+1


            if ind_concat%fps_counterPeriod==0:
                elapsed = time.time() - tic_fps
                fps = fps_counterPeriod/elapsed
                if printFPS_pref:
                    print(fps)
                tic_fps = time.time()


out.release()
video.release()
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection, neighbors)
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca',
                     random_state=0, perplexity=200)
X_tsne = tsne.fit_transform(factors_temporal)
print("Finished computing t-SNE embedding")


In [ ]:
factor_toCMap = 8  # 1 indexed

plt.figure(figsize=(5,5))
# plt.plot(X_tsne[:,0] , X_tsne[:,1] , linewidth=0.05)
# plt.scatter(X_tsne[:,0] , X_tsne[:,1], 'r.' , markersize=0.6)
plt.scatter(X_tsne[:,0] , X_tsne[:,1] , s=1.5, c=factors_temporal[:,factor_toCMap-1] , cmap='jet')


In [ ]:
# das it

In [ ]:
torch.cuda.empty_cache()